In [2]:
import pandas as pd

In [4]:
ddf = pd.read_csv('data/temps_adp20-1.csv')

In [5]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            348 non-null    int64  
 1   month           348 non-null    int64  
 2   day             348 non-null    int64  
 3   week            348 non-null    object 
 4   temp_2          348 non-null    int64  
 5   temp_1          348 non-null    int64  
 6   average         348 non-null    float64
 7   actual          348 non-null    int64  
 8   forecast_noaa   348 non-null    int64  
 9   forecast_acc    348 non-null    int64  
 10  forecast_under  348 non-null    int64  
 11  friend          348 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 32.8+ KB


In [7]:
ddf.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


In [8]:
ddf.tail()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
343,2016,12,27,Tues,42,42,45.2,47,41,50,47,47
344,2016,12,28,Wed,42,47,45.3,48,41,49,44,58
345,2016,12,29,Thurs,47,48,45.3,48,43,50,45,65
346,2016,12,30,Fri,48,48,45.4,57,44,46,44,42
347,2016,12,31,Sat,48,57,45.5,40,42,48,47,57


In [23]:
ddf['year'] = ddf['year'].astype('str')
ddf['month'] = ddf['month'].astype('str')
ddf['day'] = ddf['day'].astype('str')

In [26]:
ddf['date'] = ddf['year']+'-'+ddf['month']+'-'+ddf['day']

In [27]:
ddf.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend,date
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29,2016-1-1
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61,2016-1-2
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56,2016-1-3
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53,2016-1-4
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41,2016-1-5


In [28]:
df= ddf.copy()
df['date'] = pd.to_datetime(df['date'])

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   year            348 non-null    object        
 1   month           348 non-null    object        
 2   day             348 non-null    object        
 3   week            348 non-null    object        
 4   temp_2          348 non-null    int64         
 5   temp_1          348 non-null    int64         
 6   average         348 non-null    float64       
 7   actual          348 non-null    int64         
 8   forecast_noaa   348 non-null    int64         
 9   forecast_acc    348 non-null    int64         
 10  forecast_under  348 non-null    int64         
 11  friend          348 non-null    int64         
 12  date            348 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(7), object(4)
memory usage: 35.5+ KB


In [31]:
df.tail()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend,date
343,2016,12,27,Tues,42,42,45.2,47,41,50,47,47,2016-12-27
344,2016,12,28,Wed,42,47,45.3,48,41,49,44,58,2016-12-28
345,2016,12,29,Thurs,47,48,45.3,48,43,50,45,65,2016-12-29
346,2016,12,30,Fri,48,48,45.4,57,44,46,44,42,2016-12-30
347,2016,12,31,Sat,48,57,45.5,40,42,48,47,57,2016-12-31
